In [1]:
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import skimage
from skimage.io import imread,imsave
from skimage.filters import threshold_otsu
import numpy as np
from skimage import morphology

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [ ]:
def whole_dice_metric(y_pred,y_true):
    smooth = 10e-16
    # single image so just roll it out into a 1D array
    
    m1 =np.reshape(y_pred,(-1))/255
    m2 =np.reshape(y_true,(-1))/255
    
    
    intersection = (m1 * m2)

    score = 2. * (np.sum(intersection) + smooth) / (np.sum(m1) +(np.sum(m2) + smooth))
        
    return score

def multiple_erosion(img,iter=5):
    for j in (range(5)):
        img=morphology.binary_erosion(img, selem=morphology.selem.disk(1))
    return img

def multiple_dialte(img,iter=5):
    for j in (range(5)):
        img=morphology.binary_dilation(img, selem=morphology.selem.disk(1))
    return img


In [ ]:

img_list=os.listdir('prediction')
avg=0
for img_name in img_list:
    img_pred=imread('prediction/'+img_name)
    gt=imread('processed_data/nucleus_maps/'+img_name.split('.')[0].split('_')[1]+'_nucleus_mask.png')
    thresh=threshold_otsu(img_pred)
    img_pred=img_pred>thresh

    dice=whole_dice_metric(img_pred.astype(np.float32)*255,gt)
    avg+=dice
    print(img_name.split('.')[0].split('_')[1],' : ',dice,thresh)
print(avg/len(img_list))

TCGA-E2-A14V-01Z-00-DX1  :  0.8225290612209419<br/>
TCGA-21-5786-01Z-00-DX1  :  0.6991805205647318<br/>
TCGA-21-5784-01Z-00-DX1  :  0.7702200475923103<br/>
TCGA-B0-5710-01Z-00-DX1  :  0.7629844577597736<br/>
TCGA-G2-A2EK-01A-02-TSB  :  0.7624224458621036<br/>
TCGA-CH-5767-01Z-00-DX1  :  0.7970872142252329<br/>
TCGA-RD-A8N9-01A-01-TS1  :  0.8176449329279959<br/>
TCGA-G9-6348-01Z-00-DX1  :  0.7966429519738157<br/>
TCGA-DK-A2I6-01A-01-TS1  :  0.8649618223341795<br/>
TCGA-KB-A93J-01A-01-TS1  :  0.8129848869951398<br/>
TCGA-AY-A8YK-01A-01-TS1  :  0.7642849375366202<br/>
TCGA-E2-A1B5-01Z-00-DX1  :  0.7747906286441217<br/>
TCGA-NH-A8F7-01A-01-TS1  :  0.7865019409500186<br/>
TCGA-HE-7128-01Z-00-DX1  :  0.6759419369045772<br/>
0.77915555610654<br/>
# Model 30th Jan 2020 model_optim.pth
TCGA-21-5784-01Z-00-DX1  :  0.8116361668301593 108<br/>
TCGA-RD-A8N9-01A-01-TS1  :  0.8782868692362495 117<br/>
TCGA-CH-5767-01Z-00-DX1  :  0.815197744362616 115<br/>
TCGA-AY-A8YK-01A-01-TS1  :  0.7743249219687112 114<br/>
TCGA-G2-A2EK-01A-02-TSB  :  0.7873815363001121 108<br/>
TCGA-G9-6348-01Z-00-DX1  :  0.8273699189706346 112<br/>
TCGA-KB-A93J-01A-01-TS1  :  0.8798372531910145 118<br/>
TCGA-DK-A2I6-01A-01-TS1  :  0.883796541868124 115<br/>
TCGA-21-5786-01Z-00-DX1  :  0.7315855593758265 111<br/>
TCGA-E2-A14V-01Z-00-DX1  :  0.8599896627535161 113<br/>
TCGA-NH-A8F7-01A-01-TS1  :  0.7949021136999774 113<br/>
TCGA-E2-A1B5-01Z-00-DX1  :  0.802531826634244 110<br/>
TCGA-HE-7128-01Z-00-DX1  :  0.788953461927371 105<br/>
TCGA-B0-5710-01Z-00-DX1  :  0.828047288316124 109<br/>
0.8188457761024771<br/>

In [ ]:

img_list=os.listdir('prediction_CoNSeP')
avg=0
for img_name in img_list:
    img_pred=imread('prediction_CoNSeP/'+img_name)
    gt=np.load('other_data/CoNSeP/Test/Labels/'+'_'.join(img_name.split('_')[1:]).split('.')[0]+'.npy')[:,:,0]
    gt=(gt>0).astype(np.uint8)*255
    
    thresh=threshold_otsu(img_pred)
    img_pred=img_pred>thresh
    imsave('prediction_CoNSeP/'+img_name.split('.')[0]+'_proc.png',img_pred.astype(np.float32))

    dice=whole_dice_metric(img_pred.astype(np.float32)*255,gt)
    avg+=dice
    print(img_name.split('.')[0],' : ',dice,thresh)
print(avg/len(img_list))

In [8]:
# # os.mkdir("kumar_he")
# import cv2
# def convert_to_png(img_path):
    
#     img=cv2.imread(img_path,1)

#     img_path=img_path.replace('Tissue_images','png_images')
#     img_path=img_path.replace('.tif','.png')
#     cv2.imwrite(img_path,img)
    
# #convet_to_png()   
# for img in tqdm([x for x in os.listdir('Tissue_images/') if '.tif' in x][1:]):
# #     print('Tissue_images/'+img.split('.')[0]+'.png')
#     convert_to_png(os.path.join('Tissue_images',img))
    

100%|██████████| 29/29 [00:04<00:00,  8.42it/s]
